In [1]:

from asyncio.windows_events import NULL
import requests
import json 
from types import SimpleNamespace
import csv



print("Пример работы с API Сервиса получения данных")
BASE_URL = 'http://www.cbr.ru/dataservice/' #источник данных

print("**** Каталог ****")
response = requests.get(f"{BASE_URL}/categoryNew")
if response.status_code != 200:
    print(f"Ошибка запроса: {response.status_code}")
    quit()
    
catalogObject= response.json(object_hook=lambda d: SimpleNamespace(**d))
# for catObj in catalogObject.category:
#     print(f"id:{catObj.category_id} title:{catObj.category_name} / {catObj.indicator_name} indicator id: {catObj.indicator_id}")



print("Формируем файл каталога категорий categoryNew.csv")
fl = open('categoryNew.csv','w')
csv_file = csv.writer(fl,delimiter=";",quotechar='"',quoting=csv.QUOTE_NONNUMERIC, lineterminator="\r")
csv_file.writerow(["category_id", "category_name", "indicator_id", "indicator_parent", "indicator_name","link", "begin_dt", "end_dt"])
for item in catalogObject.category:
    csv_file.writerow([item.category_id,
                      item.category_name,
                      item.indicator_id,
                      item.indicator_parent,
                      item.indicator_name,
                      item.link,
                      item.begin_dt,
                      item.end_dt]
                      )
fl.close()


print("**** получаем данные ****")

#categoryId = 18 - Статистика процентных ставок - По депозитам - В целом по Российской Федерации
#i_ids=38 - Ставки по вкладам (депозитам) нефинансовых организаций
#m1_ids= [3,4] - выбираем данные в долларах(3) и евро(4), (параметр может отсуствовать)
#m2_ids = [9,10] - для 9 - От 1 года до 3 лет, 10 - Свыше 3 лет, (параметр может отсуствовать)
# y1 и y2 соотв. диапазон годов
# data = {'categoryId': 5, 'y1': 2021, 'y2': 2023 , 'i_ids[]': [7,8]}
# data = {'categoryId': 18, 'y1': 2021, 'y2': 2023 , 'i_ids[]': [38]} # получаем все данные для 'Ставки по вкладам (депозитам) нефинансовых организаций'

data = {'categoryId': 18, 'y1': 2021, 'y2': 2023 , 'i_ids[]': [38], 'm1_ids[]': [3,4], 'm2_ids[]': [9, 10]} 


response = requests.get(f"{BASE_URL}/dataNew", data)
if response.status_code != 200:
    print(f"Ошибка запроса: {response.status_code}")
    quit()
    

dataObject= response.json(object_hook=lambda d: SimpleNamespace(**d))
# в результате запроса получаем две таблицы dataObject.RowData - динамический ряд наблюдений со ссылками на справочники
# dataObject.Links - список справочников (справочник показателей, два справочника измерений и справочник единиц измерений)

print("Формируем файл с данными data.csv")
fl = open('data.csv','w')
csv_file = csv.writer(fl,delimiter=";",quotechar='"',quoting=csv.QUOTE_NONNUMERIC, lineterminator="\r")
csv_file.writerow(["date","periodicity", "title","m1", "m2",  "value", "dim"])

for item in dataObject.RowData:    
    
    ind_name= next(f for f in dataObject.Links if  (f.indicator_id == item.indicator_id
                                                    and 
                                                    f.measure1_id == item.measure1_id 
                                                    and
                                                    f.measure2_id == item.measure2_id
                                                    and 
                                                    f.unit_id==item.unit_id))     

    csv_file.writerow([item.date,
                       item.periodicity,
                       ind_name.indicator_name,
                       ind_name.measure1_name,
                       ind_name.measure2_name,
                       item.obs_val,                      
                       ind_name.un_name]
                      )
fl.close()



print("Все готово...")

Пример работы с API Сервиса получения данных
**** Каталог ****
Формируем файл каталога категорий categoryNew.csv
**** получаем данные ****
Формируем файл с данными data.csv
Все готово...


In [ ]:
import pandas as pd

df = pd.read_csv('categoryNew.csv', )

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc4 in position 111: invalid continuation byte

In [5]:
from asyncio.windows_events import NULL
import requests
import json 
from types import SimpleNamespace
import csv

# Пример использования методов, позволяюших получить данные сервиса одним запросом, используя массивы параметров в качестве параметров запроса.

print("Пример работы с API Сервиса получения данных с помощью методов categoryNew и dataNew ")
BASE_URL = 'http://www.cbr.ru/dataservice/' #источник данных

print("**** Каталог ****")
response = requests.get(f"{BASE_URL}/categoryNew")
if response.status_code != 200:
    print(f"Ошибка запроса: {response.status_code}")
    quit()
    
catalogObject= response.json(object_hook=lambda d: SimpleNamespace(**d))
for catObj in catalogObject.category:
    print(f"id:{catObj.category_id} title:{catObj.category_name} / {catObj.indicator_name} indicator id: {catObj.indicator_id}")



print("Формируем файл каталога показателей categoryNew.csv")
fl = open('categoryNew.csv','w')
csv_file = csv.writer(fl,delimiter=";",quotechar='"',quoting=csv.QUOTE_NONNUMERIC, lineterminator="\r")
csv_file.writerow(["category_id", "category_name", "indicator_id", "indicator_parent", "indicator_name","link", "begin_dt", "end_dt"])
for item in catalogObject.category:
    csv_file.writerow([item.category_id,
                      item.category_name,
                      item.indicator_id,
                      item.indicator_parent,
                      item.indicator_name,
                      item.link,
                      item.begin_dt,
                      item.end_dt]
                      )
fl.close()


print("**** получаем данные: "
      "пример: \"Денежно-кредитная статистика-Структура денежной массы\" - \"Денежный агрегат М2 + Широкая денежная масса\" c 2021 по 2023 "
      "****")

#categoryId = 5 - Денежно-кредитная статистика-Структура денежной массы
#i_ids=7 - Денежный агрегат М2 + + 
#i_ids=8 - Широкая денежная масса
# y1 и y2 соотв. диапазон годов

data = {'categoryId': 5, 'y1': 2021, 'y2': 2023 , 'i_ids[]': [7,8]}

response = requests.get(f"{BASE_URL}/dataNew", data)
if response.status_code != 200:
    print(f"Ошибка запроса: {response.status_code}")
    quit()
    

dataObject= response.json(object_hook=lambda d: SimpleNamespace(**d))


print("Формируем файл с данными dataM2_wm.csv")
fl = open('dataM2_wm.csv','w')
csv_file = csv.writer(fl,delimiter=";",quotechar='"',quoting=csv.QUOTE_NONNUMERIC, lineterminator="\r")
csv_file.writerow(["id", "indicator_id", "measure1_id", "measure2_id", "unit_id","obs_val", "date", "periodicity"])

for item in dataObject.RowData:
    csv_file.writerow([item.id,
                      item.indicator_id,
                      item.measure1_id,
                      item.measure2_id,
                      item.unit_id,
                      item.obs_val,
                      item.date,
                      item.periodicity]
                      )
fl.close()


print("Формируем файл с описанием dataM2_wm_link.csv")
fl = open('dataM2_wm_link.csv','w')
csv_file = csv.writer(fl,delimiter=";",quotechar='"',quoting=csv.QUOTE_NONNUMERIC, lineterminator="\r")
csv_file.writerow(["indicator_id", "indicator_parent", "measure1_id", "measure2_id", "unit_id","indicator_name", "measure1_name", "measure2_name","un_name"])

for item in dataObject.Links:
    csv_file.writerow([item.indicator_id,
                      item.indicator_parent,
                      item.measure1_id,
                      item.measure2_id,
                      item.unit_id,
                      item.indicator_name,
                      item.measure1_name,
                      item.measure2_name,
                      item.un_name]
                      )
fl.close()


print("Все готово...")

Пример работы с API Сервиса получения данных с помощью методов categoryNew и dataNew 
**** Каталог ****
id:5 title:Денежно-кредитная статистика-Структура денежной массы / Денежный агрегат М0 indicator id: 5
id:5 title:Денежно-кредитная статистика-Структура денежной массы / Денежный агрегат М1 indicator id: 6
id:5 title:Денежно-кредитная статистика-Структура денежной массы / Денежный агрегат М2 indicator id: 7
id:5 title:Денежно-кредитная статистика-Структура денежной массы / Широкая денежная масса indicator id: 8
id:8 title:Статистика внешнего сектора-Платежный баланс Российской Федерации-Ключевые агрегаты / Сальдо счета операций с капиталом indicator id: 10
id:8 title:Статистика внешнего сектора-Платежный баланс Российской Федерации-Ключевые агрегаты / Сальдо счета текущих операций indicator id: 9
id:8 title:Статистика внешнего сектора-Платежный баланс Российской Федерации-Ключевые агрегаты / Сальдо финансового счета indicator id: 11
id:8 title:Статистика внешнего сектора-Платежный ба

In [34]:
df_2 = pd.read_csv('dataM2_wm_link.csv', encoding='cp1251', sep=';')

In [28]:
import datetime
from datetime import datetime

In [ ]:
# df_2['date'] = pd.to_datetime(df_2['date'])

In [35]:
df_2

,indicator_id,indicator_parent,measure1_id,measure2_id,unit_id,indicator_name,measure1_name,measure2_name,un_name
0,7,-1,NaN,12,3,Денежный агрегат М2,NaN,Всего,млрд руб.
1,7,-1,NaN,19,3,Денежный агрегат М2,NaN,Денежный агрегат М1,млрд руб.
2,7,-1,NaN,20,3,Денежный агрегат М2,NaN,Другие депозиты других финансовых организаций,млрд руб.
3,7,-1,NaN,21,3,Денежный агрегат М2,NaN,Другие депозиты нефинансовых организаций,млрд руб.
4,7,-1,NaN,22,3,Денежный агрегат М2,NaN,Другие депозиты домашних хозяйств,млрд руб.
5,8,-1,NaN,12,3,Широкая денежная масса,NaN,Всего,млрд руб.
6,8,-1,NaN,23,3,Широкая денежная масса,NaN,Денежный агрегат М2,млрд руб.
7,8,-1,NaN,24,3,Широкая денежная масса,NaN,Депозиты в иностранной валюте других финансовы...,млрд руб.
8,8,-1,NaN,25,3,Широкая денежная масса,NaN,Депозиты в иностранной валюте нефинансовых орг...,млрд руб.
9,8,-1,NaN,26,3,Широкая денежная масса,NaN,Депозиты в иностранной валюте домашних хозяйств,млрд руб.
